In [2]:
import pandas as pd
import altair as alt 
from vega_datasets import data, local_data

## Aplicando connhecimento e diferentes funcionalidades

Nessa seção, nosso objetivo foi apresentar exemplos aplicando os aprendizados do capítulo juntamente com outras ferramentas das bibliotecas Altair e Pandas. Tais gráficos visam a ampliar o repertório do nosso leitor, otimizar seu aprendizado e instigar sua curiosidade. Os exemplos abaixo foram construídos a partir de diferentes *datasets* do repositório [vega_datasets](https://github.com/vega/vega-datasets).

### Trabalhando com *datasets* da coleção vega_datasets

Antes de escolher o *dataset* que iremos trabalhar, iremos ressaltar algumas propriedades interessantes do pacote vega_datasets.

Para listar todos os *datasets* disponíveis, use o método `list_datasets`:

In [3]:
from vega_datasets import local_data
local_data.list_datasets()

['airports',
 'anscombe',
 'barley',
 'burtin',
 'cars',
 'crimea',
 'driving',
 'iowa-electricity',
 'iris',
 'la-riots',
 'ohlc',
 'seattle-temps',
 'seattle-weather',
 'sf-temps',
 'stocks',
 'us-employment',
 'wheat']

Se quiser obter mais informações sobre qualquer *dataset*, você pode usar a propriedade `description`:

In [4]:
data.cars.description

'Acceleration, horsepower, fuel efficiency, weight, and other characteristics of different makes and models of cars. This dataset was originally published by Donoho et al (1982) [1]_, and was made public at http://lib.stat.cmu.edu/datasets/'

### O *Dataset* Iris

Para esse exemplo usaremos o *dataset* `iris` do repositório vega_datasets. Começaremos olhando a descrição desse *dataset*:

In [5]:
data.iris.description

'This classic dataset contains lengths and widths of petals and sepals for 150 iris flowers, drawn from three species. It was introduced by R.A. Fisher in 1936 [1]_.'

"Esse conjunto de dados clássico contém comprimentos e larguras de pétalas e sépalas de 150 flores de íris, provenientes de três espécies. Ele foi introduzido por R.A. Fisher em 1936 [1]_."

Essa descrição nos permite ter uma noção melhor de quais variáveis temos disponíveis, quais os tipos de dados que podemos trabalhar, o tamanho do *data frame*, etc. Vamos então importar esse *dataset* para comerçarmos a análise:

In [6]:
iris = data.iris()
iris.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Assim, como mostrado no notebook anterior, podemos renomear as colunas para melhor entendimento, usando `rename(columns={})`.

In [7]:
iris = iris.rename(columns={
    'sepalLength': 'Comprimento da Sépala',
    'sepalWidth': 'Largura da Sépala',
    'petalLength': 'Comprimento da Pétala',
    'petalWidth': 'Largura da Pétala',
    'species': 'Espécie'
})
iris.head()

,Comprimento da Sépala,Largura da Sépala,Comprimento da Pétala,Largura da Pétala,Espécie
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Sobre os tipos de dados, temos dados quantitivos de largura e comprimento, tanto da pétala quando da sépala, e dados nominais da espécie.

Podemos também definir um esquema de cores específico para os gráficos, para combinar melhor com o tema e contexto. Lembrando que as cores para os gráficos que optamos por fazer abaixo, não devem formar gradiente ou expressar noção de hierarquia, uma vez que essa relação não existe, e não é explorada, entre as espécies trabalhadas. Para saber melhor como é feito, consulte o notebook anterior.

In [8]:
color_map = alt.Scale(
    domain = ['setosa', 'versicolor', 'virginica'],
    range = ['#8FBED6', '#93C572', '#8A2BE2']
)

Vamos começar fazendo gráficos de dispersão relacionando as diferentes variáveis.

In [10]:
disp1 = alt.Chart(iris).mark_point(filled=True).encode(
    alt.X('Comprimento da Pétala',),
    alt.Y('Comprimento da Sépala'),
    alt.Color('Espécie:N', scale=color_map)
).properties(
    width=200,
    height=200
)

disp2 = alt.Chart(iris).mark_point(filled=True).encode(
    alt.X('Largura da Pétala'),
    alt.Y('Largura da Sépala'),
    alt.Color('Espécie:N', scale=color_map)
).properties(
    width=200,
    height=200
)

disp3 = alt.Chart(iris).mark_point(filled=True).encode(
    alt.X('Comprimento da Pétala'),
    alt.Y('Largura da Pétala'),
    alt.Color('Espécie:N', scale=color_map)
).properties(
    width=200,
    height=200
)

disp4 = alt.Chart(iris).mark_point(filled=True).encode(
    alt.X('Comprimento da Sépala'),
    alt.Y('Largura da Sépala'),
    alt.Color('Espécie:N', scale=color_map)
).properties(
    width=200,
    height=200
)

alt.hconcat(disp1, disp2, disp3, disp4)

c:\Users\bianc\AppData\Local\Microsoft\WindowsApps\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

Podemos observar que nos três primeiros gráficos, as espécies *versicolor* e *virginica* se encontram próxima e em partes sobrepostas na distrubuição. Já a espécie *setosa*, se comporta como um cluster a parte. 

Podemos analisar a distribução com um gráfico de estimativa de densidade.

In [11]:
alt.Chart(iris, height=100).transform_fold(
    [
        "Largura da Pétala",
        "Comprimento da Pétala",
        "Largura da Sépala",
        "Comprimento da Sépala",
    ],
    as_=["Tipo de Medida", "value"],
).transform_density(
    density="value",
    bandwidth=0.3,
    groupby=["Tipo de Medida"],
    extent=[0, 8],
).mark_area(color='#8A2BE2').encode(
    alt.X("value:Q", title="Valor"),
    alt.Y("density:Q", title='Densidade'),
    alt.Row("Tipo de Medida:N"),
).properties(
    width=300
)

c:\Users\bianc\AppData\Local\Microsoft\WindowsApps\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Unindo as informações do gráfico de dispersão, para as características da Pétala e da Sépala, e o gráfico de distribuição de densidade, podemos concluir que para as característica da Pétala, temos uma densidade divida em dois picos e em dois clusters. Podemos analisar isso mais a fundo com outros tipos de gráficos, como a seguir.

Se tratando de tipos de dados quantitatiivos, como temos nesse *dataset*, uma visualização interssante seria o *boxplot*. Ainda, se estivermos interessados na distribuição de probabilidade junto aos outros dados que o *boxplot* nos fornece, como mediana, limites superior e inferior, outliers, etc., podemos usar o gráfico de violino para obter uma combinação dessas informações.

In [12]:
def violin_plot(record: str):
    plot = alt.Chart(iris).transform_density(
    record,
    as_=[record, 'density'],
    extent=[0, 9],
    groupby=['Espécie']
    ).mark_area(orient='horizontal').encode(
        alt.X('density:Q')
            .stack('center')
            .impute(None)
            .title(None)
            .axis(labels=False, values=[0], grid=False, ticks=True),
        alt.Y(f'{record}:Q'),
        alt.Color('Espécie:N', scale=color_map),
        alt.Column('Espécie:N')
            .spacing(0)
            .header(titleOrient='bottom', labelOrient='bottom', labelPadding=0)
    ).properties(
        width=70,
        height=200
    )
    return plot

comp_sepala = violin_plot('Comprimento da Sépala')

larg_sepala = violin_plot('Largura da Sépala')

comp_petala = violin_plot('Comprimento da Pétala')

larg_petala = violin_plot('Largura da Pétala')

alt.hconcat(comp_sepala, larg_sepala, comp_petala, larg_petala).configure_view(stroke=None)

c:\Users\bianc\AppData\Local\Microsoft\WindowsApps\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

Podemos observar que nas características da pétala, a distribuição da espécie *setosa* se concentra expressivamente próximo da média. 

Podemos verificar isso mais detalhadamente com um simples histrograma. Para isso, criaremos um *data frame* apenas com os dados dessa espécie para as características da pétala.

In [13]:
iris_setosa = iris.loc[iris['Espécie'] == 'setosa']
setosa_petalas = iris_setosa[['Comprimento da Pétala', 'Largura da Pétala', 'Espécie']]
setosa_petalas.head()

,Comprimento da Pétala,Largura da Pétala,Espécie
0,1.4,0.2,setosa
1,1.4,0.2,setosa
2,1.3,0.2,setosa
3,1.5,0.2,setosa
4,1.4,0.2,setosa


Criamos então um histograma, usando o método `mark_bar()` e alteramos a cor do plot com a propriedade `color`, alteramos a cor para corresponder mais ao contexto de flores de iris (passo opicional). Para visualizar se os valores se concentram em torno da média, sobrepomos uma linha demarcando esse valor, usando o método `mark_rule()` e definindo o canal `x='mean(Comprimento da Pétala):Q'`.

In [14]:
comp_hist = alt.Chart(setosa_petalas).mark_bar(color='#8fbed6').encode(
    alt.X('Comprimento da Pétala:Q').bin().axis(None),
    y='count()'
) # Cria as caixas do histograma para o Comprimento da Pétala

comp_hist_rule = alt.Chart(setosa_petalas).mark_rule(color='#8A2BE2').encode(
    x='mean(Comprimento da Pétala):Q',
    size=alt.value(5)
) # Cria a régua que demarca a média

comp = (comp_hist + comp_hist_rule).properties(
    title = 'Distribuição do Comprimento da Pétala na espécie Setosa' # Adiciona um título ao gráfico
) # Une os dois elementos em um único gráfico

larg_hist = alt.Chart(setosa_petalas).mark_bar(color='#8fbed6').encode(
    alt.X('Largura da Pétala:Q').bin().axis(None),
    y='count()'
) # Cria as caixas do histograma para a Largura da Pétala

larg_hist_rule = alt.Chart(setosa_petalas).mark_rule(color='#8A2BE2').encode(
    x='mean(Largura da Pétala):Q',
    size=alt.value(5)
) # Cria a régua que demarca a média

larg = (larg_hist + larg_hist_rule).properties(
    title = 'Distribuição do Largura da Pétala na espécie Setosa' # Adiciona um título ao gráfico
) # Une os dois elementos em um único gráfico

# Concatena horizontalmente os dois gráficos criados para exibição
comp | larg 

c:\Users\bianc\AppData\Local\Microsoft\WindowsApps\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

Como podemos ver, a hipótese é confirmada e os valores se concentram em torno da média, principalmente para a Largura da Pétala.